Задание
Рассчитайте наценку на ювелирные изделия по данным на листе "Задание 2". Соблюсти следующие условия:
1. Период анализа - с 1.04.23 по 10.04.23
2. В анализе должны участвовать только продажи изделий с общей суммой и себестоимостью больше 0 рублей
3. В ответе необходимо получить два значения наценки - среднюю наценку на ювелирные изделия из золота и среднюю наценку на ювелирные изделия из серебра. Сгруппировать изделия по металлу нужно на основе поля "Товарная группа". Товарная группа изделий из серебра начинается на "СИ", все остальные изделия - это золото.

# Обработка данных

In [81]:
import numpy as np
import pandas as pd

In [82]:
df = pd.read_csv("/content/data.csv")
df.head(6)

,ДатаПродажи,НомерЧека,Номенклатура,Штрихкод,ТоварнаяГруппа,СН_Характеристика,бгсВес,ОбщаяСумма,Себестоимость
0,4/1/23 10:05,1814,серьги с брил.+изумрудом с родир,1078601904686,ДК СЕРЬГИ,Au 585º -,"2,89",51180,"17276,0475"
1,4/1/23 10:08,4868,Серьги с топазом,1078551306013,ПДК СЕРЬГИ,Au 585º -,"3,99",34740,"12074,747"
2,4/1/23 10:08,617,Кольцо с брилл.+изум.гидротерм. с родир.,1078509166518,ДК КОЛЬЦА,"Au 585º -, разм.18,5","2,05",17760,"6629,612"
3,4/1/23 10:11,461,Кольцо с амет. род.,1078579406088,СИ КОЛЬЦО ЦВ/К,"Ag 925º -, разм.19,0","1,46",756,"302,22"
4,4/1/23 10:12,3939,Цепь Якорь с а/г род.,1078601866814,СИ ЦЕПЬ,"Ag 925º -, разм.40,0","2,7",1560,"458,9995"
5,4/1/23 10:13,3194,Браслет с фиан. поз.,1078540181198,СИ БРАСЛЕТ СК,"Ag 925º -, разм.19,0","4,98","4622,4","1300,029"


In [83]:
print(df.dtypes)

ДатаПродажи          object
НомерЧека             int64
Номенклатура         object
Штрихкод              int64
ТоварнаяГруппа       object
СН_Характеристика    object
бгсВес               object
ОбщаяСумма           object
Себестоимость        object
dtype: object


In [85]:
df['бгсВес'] = pd.to_numeric(df['бгсВес'], errors='coerce')
df['Себестоимость'] = pd.to_numeric(df['Себестоимость'], errors='coerce')
df['ОбщаяСумма'] = pd.to_numeric(df['ОбщаяСумма'], errors='coerce')
df['ДатаПродажи'] = pd.to_datetime(df['ДатаПродажи'], format='%m/%d/%y %H:%M')

In [86]:
print(df.dtypes)

ДатаПродажи          datetime64[ns]
НомерЧека                     int64
Номенклатура                 object
Штрихкод                      int64
ТоварнаяГруппа               object
СН_Характеристика            object
бгсВес                      float64
ОбщаяСумма                  float64
Себестоимость               float64
dtype: object


In [87]:
df = df.dropna()

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, 585 to 139456
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ДатаПродажи        82 non-null     datetime64[ns]
 1   НомерЧека          82 non-null     int64         
 2   Номенклатура       82 non-null     object        
 3   Штрихкод           82 non-null     int64         
 4   ТоварнаяГруппа     82 non-null     object        
 5   СН_Характеристика  82 non-null     object        
 6   бгсВес             82 non-null     float64       
 7   ОбщаяСумма         82 non-null     float64       
 8   Себестоимость      82 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 6.4+ KB


# Фильтрация, группировка

In [89]:
df['ДатаПродажи'] = pd.to_datetime(df['ДатаПродажи'], format='%d/%m/%y %H:%M')

filtered_df = df[
    (df['ДатаПродажи'] >= '2023-04-01') &
    (df['ДатаПродажи'] <= '2023-04-11')
]

In [90]:
filtered_df = filtered_df[
    (filtered_df["Себестоимость"] > 0) & (filtered_df["ОбщаяСумма"] > 0)
].copy()

In [91]:
filtered_df['Материал'] = filtered_df['ТоварнаяГруппа'].str.startswith('СИ').map({True: 'Серебро', False: 'Золото'})

In [92]:
print(f'После фильтрации по дате: {len(filtered_df)} строк')

После фильтрации по дате: 62 строк


In [93]:
filtered_df['Наценка'] = ((filtered_df['ОбщаяСумма']- filtered_df['Себестоимость'])/filtered_df['Себестоимость']) * 100

In [80]:
filtered_df.head()

,ДатаПродажи,НомерЧека,Номенклатура,Штрихкод,ТоварнаяГруппа,СН_Характеристика,бгсВес,ОбщаяСумма,Себестоимость,Материал,Наценка
585,2023-04-01 14:24:00,4951,Кольцо с фиан. род.,1078540069111,СИ КОЛЬЦО ИФ,"Ag 925º -, разм.19,0",4.0,2952.0,1242.0,Серебро,137.681159
794,2023-04-01 15:48:00,1866,Подвес с фиан. род.,1078585671859,СИ ПОДВЕС ИФ,Ag 925º -,1.0,780.0,276.0,Серебро,182.608696
1588,2023-04-02 12:05:00,1145,Цепь Сингапур с а/г,1078576388171,СИ ЦЕПЬ,"Ag 925º -, разм.50,0",4.0,1620.0,437.0,Серебро,270.709382
3034,2023-04-03 14:09:00,416,Подвес шарм Перо с черн.,1078601118138,СИ ШАРМЫ,Ag 925º -,1.0,324.0,161.0,Серебро,101.242236
3813,2023-04-04 11:55:00,4770,Цепь Снейк мягкий 8 граней род.,1078511134396,СИ ЦЕПЬ,"Ag 925º -, разм.40,0",3.0,1668.0,621.0,Серебро,168.599034


# Средние наценки по материалу

In [94]:
mean_serebro = round(filtered_df[filtered_df['Материал'] == 'Серебро']['Наценка'].mean(), 2)
mean_zoloto = round(filtered_df[filtered_df['Материал'] == 'Золото']['Наценка'].mean(), 2)

print (f'Средняя наценка на серебро ', mean_serebro, '%')
print (f'Средняя наценка на золото ', mean_zoloto, '%')

Средняя наценка на серебро  214.38 %
Средняя наценка на золото  118.45 %
